In [2]:
import pandas as pd

In [6]:
class machine():
    
    def __init__(self, p, inp):
        self.p = p + [0] * 1000
        self.inputs = [inp]
    
    def add_input(self, inp):
        self.inputs.append(inp)
    
    def run(self):
        p = self.p
        rel_base = 0
        def get_param(ip, modes, n):
            addr = p[ip+n]
            if modes[n-1]==0: return p[addr]              # position mode
            elif modes[n-1]==1: return addr               # immediate mode
            elif modes[n-1]==2: return p[rel_base + addr] # relative mode
            else:
                print("Unknown relative mode for read")
                return
        def set_param(ip, modes, n, val):
            addr = p[ip+n]
            if modes[n-1]==0: p[addr] = val
            elif modes[n-1]==2: p[rel_base + addr] = val
            else:
                print("Unknown relative mode for write")
        ip = 0
        while True:
            first = p[ip]
            op = first % 100
            if op == 99: return
            modes = list(map(int, str(first // 100)))
            if op in [1,2,7,8]: param_count = 3
            elif op in [5,6]: param_count = 2
            elif op in [3,4,9]: param_count = 1
            else:
                print("Unknown opcode", op)
                return
            while len(modes) < param_count: modes = [0] + modes
            modes = modes[::-1]
            if op == 3:
                set_param(ip, modes, 1, self.inputs[0])
                self.inputs = self.inputs[1:]
            elif op == 4: yield get_param(ip, modes, 1)
            elif op == 9: rel_base += get_param(ip, modes, 1)
            elif op == 1: set_param(ip, modes, 3, get_param(ip, modes, 1) + get_param(ip, modes, 2))
            elif op == 2: set_param(ip, modes, 3, get_param(ip, modes, 1) * get_param(ip, modes, 2))
            elif op == 5:
                if get_param(ip, modes, 1)!=0:
                    ip = get_param(ip, modes, 2)
                    continue
            elif op == 6:
                if get_param(ip, modes, 1)==0:
                    ip = get_param(ip, modes, 2)
                    continue
            elif op == 7:
                if get_param(ip, modes, 1) < get_param(ip, modes, 2): set_param(ip, modes, 3, 1)
                else: set_param(ip, modes, 3, 0)
            elif op == 8:
                if get_param(ip, modes, 1) == get_param(ip, modes, 2): set_param(ip, modes, 3, 1)
                else: set_param(ip, modes, 3, 0)
            ip += 1 + param_count     
    
    def start(self):
        return self.run()

In [2]:
with open('/Users/felixpuetsch/Downloads/input11.txt', 'rt') as f:
    prog = f.read()
    prog = list(map(int, prog.strip().split(',')))

In [42]:
from collections import defaultdict

def run_robot():
    path = defaultdict(int)
    x,y = 0,0
    dx,dy = 0,1
    m = machine(prog, 1)
    r = m.start()
    while True:
        try:
            c, t = next(r), next(r)
            #print(f"x={x} y={y} color={c} turn={t}")
            path[(x,y)] = c
            if t==0: dx, dy = -dy, dx 
            else: dx, dy = dy, -dx
            x += dx
            y += dy
            m.add_input(path[(x,y)])
        except StopIteration: break
    return path

In [48]:
path = run_robot()

In [51]:
for y in range(0,-8,-1):
    s = ""
    for x in range(50):
        c = "▓" if path[(x,y)]==1 else " "
        s+=c
    print(s)

   ▓▓ ▓  ▓  ▓▓  ▓▓▓  ▓▓▓   ▓▓   ▓▓  ▓  ▓          
    ▓ ▓  ▓ ▓  ▓ ▓  ▓ ▓  ▓ ▓  ▓ ▓  ▓ ▓  ▓          
    ▓ ▓▓▓▓ ▓  ▓ ▓  ▓ ▓▓▓  ▓    ▓    ▓  ▓          
    ▓ ▓  ▓ ▓▓▓▓ ▓▓▓  ▓  ▓ ▓ ▓▓ ▓    ▓  ▓          
 ▓  ▓ ▓  ▓ ▓  ▓ ▓ ▓  ▓  ▓ ▓  ▓ ▓  ▓ ▓  ▓          
  ▓▓  ▓  ▓ ▓  ▓ ▓  ▓ ▓▓▓   ▓▓▓  ▓▓   ▓▓           
                                                  
                                                  


In [44]:
len(path.keys())

250